In [1]:
#Run to install dependencies
!pip install langchain langchain-openai langchain-groq langchain-community requests pandas yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Fou

In [3]:
#Module Imports
import os
import requests
import yfinance as yf
from langchain.tools import Tool
from langchain.agents import AgentType, initialize_agent
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
import pandas as pd
import json
import time
from datetime import datetime, timedelta

In [2]:
#Add your API keys here
os.environ['GROQ_API_KEY'] = ''
os.environ['OPENAI_API_KEY'] = ''

In [4]:
#Set up API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [27]:
#Initialize different LLMs
openai_llm = ChatOpenAI(
    model_name="gpt-4",
    temperature=0.7,
    api_key=OPENAI_API_KEY
)

groq_llm_llama3_8b = ChatGroq(
    model_name="llama3-8b-8192",
    temperature=0.7,
    api_key=GROQ_API_KEY
)

groq_llm_llama3_70b = ChatGroq(
    model_name="llama3-70b-8192",
    temperature=0.7,
    api_key=GROQ_API_KEY
)

In [28]:
#Function to select LLM based on user preference
def get_llm(provider="openai", model="default"):
    if provider.lower() == "openai":
        return openai_llm
    elif provider.lower() == "groq":
        if model.lower() == "llama3-70b":
            return groq_llm_llama3_70b
        else:
            return groq_llm_llama3_8b
    else:
        # Default to OpenAI if provider not recognized
        return openai_llm

In [36]:
# Task 1: Implement Stock Price Lookup Tool
def get_stock_price(symbol):
    try:
        if isinstance(symbol, (set, list, tuple)):
            if len(symbol) > 0:
                symbol = list(symbol)[0]
            else:
                return "Error: No valid symbol provided"

        ticker = yf.Ticker(symbol)
        ticker_info = ticker.info
        current_price = ticker_info.get('regularMarketPrice', ticker_info.get('currentPrice', None))
        hist_data = ticker.history(period="2d")

        if len(hist_data) >= 2:
            prev_close = hist_data['Close'].iloc[-2]
            if current_price is None:
                current_price = hist_data['Close'].iloc[-1]
            daily_change = current_price - prev_close
            percent_change = (daily_change / prev_close) * 100
        else:
            prev_close = None
            daily_change = None
            percent_change = None

        result = {
            "symbol": symbol,
            "current_price": current_price,
            "daily_change": daily_change,
            "percent_change": percent_change,
            "previous_close": prev_close,
            "currency": ticker_info.get('currency', 'USD'),
            "company_name": ticker_info.get('shortName', symbol),
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        return result

    except Exception as e:
        return f"Error retrieving stock price for {symbol}: {str(e)}"

# Create the stock price lookup tool
stock_price_tool = Tool(
    name="StockPriceLookup",
    func=get_stock_price,
    description="Useful for getting the current price of a stock. Input should be a valid stock ticker symbol."
)

In [43]:
# Task 2: Implement Portfolio Rebalancing Tool
def rebalance_portfolio(portfolio_str):
    try:
        if isinstance(portfolio_str, str):
            try:
                portfolio = eval(portfolio_str)
            except (SyntaxError, NameError):
                cleaned_str = portfolio_str.strip()
                if cleaned_str.startswith('{'):
                    cleaned_str = cleaned_str[1:]
                if cleaned_str.endswith('}'):
                    cleaned_str = cleaned_str[:-1]
                portfolio = {}
                pairs = [pair.strip() for pair in cleaned_str.split(',')]
                for pair in pairs:
                    if ':' in pair:
                        key, value = pair.split(':', 1)
                        key = key.strip().strip('"\'')
                        value = value.strip()
                        try:
                            portfolio[key] = float(value)
                        except ValueError:
                            return f"Error: Could not convert {value} to a number"
        else:
            portfolio = portfolio_str

        num_stocks = len(portfolio)
        target_weight = 1.0 / num_stocks

        stock_values = {}
        total_value = 0
        for symbol, weight in portfolio.items():
            price_data = get_stock_price(symbol)

            if isinstance(price_data, str) and price_data.startswith("Error"):
                return f"Error getting price for {symbol}: {price_data}"

            price = price_data["current_price"]
            portfolio_value = 100000 #Assumption
            value = float(weight) * portfolio_value
            shares = value / price

            stock_values[symbol] = {
                "weight": float(weight),
                "price": price,
                "value": value,
                "shares": shares,
                "current_weight": weight
            }
            total_value += value

        for symbol in stock_values:
            stock_values[symbol]["current_weight"] = stock_values[symbol]["value"] / total_value

        rebalance_actions = {}

        for symbol, data in stock_values.items():
            current_weight = data["current_weight"]
            weight_diff = target_weight - current_weight
            value_diff = weight_diff * total_value
            shares_diff = value_diff / data["price"]

            action = "BUY" if shares_diff > 0 else "SELL"

            rebalance_actions[symbol] = {
                "current_weight": current_weight,
                "target_weight": target_weight,
                "weight_diff": weight_diff,
                "value_diff": abs(value_diff),
                "shares": abs(shares_diff),
                "action": action
            }

        output = []
        output.append(f"Portfolio Total Value: ${total_value:.2f}")
        output.append(f"Target Weight per Stock: {target_weight:.4f} ({target_weight*100:.2f}%)")

        output.append("\nCurrent Portfolio Status:")
        for symbol, data in stock_values.items():
            output.append(f"  {symbol}: ${data['value']:.2f} ({data['current_weight']*100:.2f}% vs target {target_weight*100:.2f}%)")

        output.append("\nRecommended Actions:")
        for symbol, action in rebalance_actions.items():
            if abs(action["weight_diff"]) < 0.001:
                output.append(f"  {symbol}: No action required (already at target weight)")
            else:
                output.append(
                    f"  {symbol}: {action['action']} {action['shares']:.2f} shares (${action['value_diff']:.2f}) " +
                    f"to adjust from {action['current_weight']*100:.2f}% to {action['target_weight']*100:.2f}%"
                )
        return "\n".join(output)

    except Exception as e:
        return f"Error analyzing portfolio: {str(e)}"

# Create the rebalancer tool
rebalance_tool = Tool(
    name="PortfolioRebalancer",
    func=rebalance_portfolio,
    description="Analyzes a portfolio and suggests rebalancing actions. Input should be a dictionary mapping stock symbols to their current weight in the portfolio."
)

In [56]:
# Task 3: Implement Market Trend Analysis Tool
def market_trend_analysis():
    try:
        indices = ["^GSPC", "^DJI", "^IXIC", "^RUT"]
        index_names = {
            "^GSPC": "S&P 500",
            "^DJI": "Dow Jones Industrial Average",
            "^IXIC": "NASDAQ Composite",
            "^RUT": "Russell 2000"
        }

        end_date = datetime.now()
        start_date = end_date - timedelta(days=7)

        trends = []
        summary_data = {}

        for idx in indices:
            data = yf.download(idx, start=start_date, end=end_date, progress=False)
            if data.empty:
                trends.append(f"No data available for {index_names.get(idx, idx)}")
                continue

            if len(data) >= 1:
                first_close = data['Close'].iloc[0].item()
                last_close = data['Close'].iloc[-1].item()

                period_change = last_close - first_close
                period_change_pct = (period_change / first_close) * 100

                daily_returns = data['Close'].pct_change().dropna()
                volatility = 0
                if not daily_returns.empty:
                    volatility = daily_returns.std().item() * 100

                highest = data['High'].max().item()
                lowest = data['Low'].min().item()

                summary_data[idx] = {
                    "name": index_names.get(idx, idx),
                    "last_close": last_close,
                    "period_change": period_change,
                    "period_change_pct": period_change_pct,
                    "volatility": volatility,
                    "highest": highest,
                    "lowest": lowest
                }

        analysis = ["Market Trends Over the Past Week:"]

        for idx, data in summary_data.items():
            direction = "up" if data["period_change"] > 0 else "down"
            analysis.append(f"\n{data['name']}:")
            analysis.append(f"  Last Close: ${data['last_close']:.2f}")
            analysis.append(f"  Weekly Change: {direction} {abs(data['period_change_pct']):.2f}% (${abs(data['period_change']):.2f})")
            analysis.append(f"  Volatility: {data['volatility']:.2f}%")
            analysis.append(f"  Range: ${data['lowest']:.2f} - ${data['highest']:.2f}")

        positive_indices = sum(1 for data in summary_data.values() if data["period_change"] > 0)
        negative_indices = len(summary_data) - positive_indices

        if positive_indices > negative_indices:
            sentiment = "Overall Market Sentiment: Positive - Most major indices showed gains over the past week."
        elif positive_indices < negative_indices:
            sentiment = "Overall Market Sentiment: Negative - Most major indices showed losses over the past week."
        else:
            sentiment = "Overall Market Sentiment: Mixed - Equal number of major indices showed gains and losses."

        analysis.append(f"\n{sentiment}")
        return "\n".join(analysis)

    except Exception as e:
        return f"Error analyzing market trends: {str(e)}"

# Create the market trend analysis tool
trend_tool = Tool(
    name="MarketTrendAnalyzer",
    func=lambda _=None: market_trend_analysis(),  # Use a lambda wrapper to ignore any input
    description="Provides an analysis of recent market trends. No input required."
)

In [57]:
#Function to create and run an agent with the selected LLM
def create_and_run_agent(llm_provider="openai", llm_model="default", query=""):
    start_time = time.time()

    llm = get_llm(llm_provider, llm_model)

    tools = [stock_price_tool, rebalance_tool, trend_tool]

    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

    response = agent.run(query)
    end_time = time.time()

    execution_time = end_time - start_time
    return {
        "response": response,
        "execution_time": execution_time,
        "llm_provider": llm_provider,
        "llm_model": llm_model
    }

In [ ]:
#Function to generate performance comparison
def generate_performance_comparison(results):
    comparison_data = []
    for result in results:
        llm_name = f"{result['llm_provider'].capitalize()} {result['llm_model'] if result['llm_model'] != 'default' else 'GPT-4'}"
        execution_time = result['execution_time']
        comparison_data.append({
            "LLM": llm_name,
            "Execution Time (s)": round(execution_time, 2),
            "Response Length (chars)": len(result['response'])
        })

    comparison_df = pd.DataFrame(comparison_data)
    print("\n===== LLM Basic Performance Comparison =====")
    print(comparison_df.to_string(index=False))

In [58]:
#Test cases with different LLMs
def run_test_cases():
    user_portfolio_1 = {"AAPL": 0.50, "TSLA": 0.30, "GOOGL": 0.20}
    user_portfolio_2 = {"MSFT": 0.25, "NVDA": 0.25, "AMZN": 0.25, "META": 0.25}

    results = []

    print("Running tests with different LLMs")

    #Testing each tool separately
    print("\nTesting individual tools:")

    #Test stock price lookup
    print("Testing Stock Price Lookup")
    stock_result = get_stock_price("AAPL")
    print(f"AAPL Stock Price: {stock_result}")

    #Test portfolio rebalancer
    print("\nTesting Portfolio Rebalancer")
    rebalance_result = rebalance_portfolio(user_portfolio_1)
    print(f"Rebalance result: {rebalance_result}")

    #Test market trend analyzer
    print("\nTesting Market Trend Analyzer")
    trend_result = market_trend_analysis()
    print(f"Market trends: {trend_result}")

    #Test with OpenAI
    print("\nOpenAI GPT-4 Results:")
    try:
        p1_result = create_and_run_agent("openai", "default", f"Analyze this portfolio and suggest rebalancing: {user_portfolio_1}")
        print(f"Portfolio 1: {p1_result['response']}")
        print(f"Execution time: {p1_result['execution_time']:.2f} seconds")
        results.append(p1_result)

        p2_result = create_and_run_agent("openai", "default", f"Analyze this portfolio and suggest rebalancing: {user_portfolio_2}")
        print(f"Portfolio 2: {p2_result['response']}")
        print(f"Execution time: {p2_result['execution_time']:.2f} seconds")
        results.append(p2_result)
    except Exception as e:
        print(f"Error with OpenAI tests: {str(e)}")

    #Test with Groq LLaMA3-8B
    print("\nGroq LLaMA3-8B Results:")
    try:
        p1_result = create_and_run_agent("groq", "llama3-8b", f"Analyze this portfolio and suggest rebalancing: {user_portfolio_1}")
        print(f"Portfolio 1: {p1_result['response']}")
        print(f"Execution time: {p1_result['execution_time']:.2f} seconds")
        results.append(p1_result)

        p2_result = create_and_run_agent("groq", "llama3-8b", f"Analyze this portfolio and suggest rebalancing: {user_portfolio_2}")
        print(f"Portfolio 2: {p2_result['response']}")
        print(f"Execution time: {p2_result['execution_time']:.2f} seconds")
        results.append(p2_result)
    except Exception as e:
        print(f"Error with Groq LLaMA3-8B tests: {str(e)}")

    #Test with Groq LLaMA3-70B
    print("\nGroq LLaMA3-70B Results:")
    try:
        p1_result = create_and_run_agent("groq", "llama3-70b", f"Analyze this portfolio and suggest rebalancing: {user_portfolio_1}")
        print(f"Portfolio 1: {p1_result['response']}")
        print(f"Execution time: {p1_result['execution_time']:.2f} seconds")
        results.append(p1_result)

        p2_result = create_and_run_agent("groq", "llama3-70b", f"Analyze this portfolio and suggest rebalancing: {user_portfolio_2}")
        print(f"Portfolio 2: {p2_result['response']}")
        print(f"Execution time: {p2_result['execution_time']:.2f} seconds")
        results.append(p2_result)
    except Exception as e:
        print(f"Error with Groq LLaMA3-70B tests: {str(e)}")

    if results:
        generate_performance_comparison(results)
    else:
        print("\nNo results available for comparison.")

In [54]:
if __name__ == "__main__":
    run_test_cases()

Running tests with different LLMs...

Testing individual tools:
Testing Stock Price Lookup...
AAPL Stock Price: {'symbol': 'AAPL', 'current_price': 198.85, 'daily_change': np.float64(26.430001831054682), 'percent_change': np.float64(15.328849386227988), 'previous_close': np.float64(172.4199981689453), 'currency': 'USD', 'company_name': 'Apple Inc.', 'timestamp': '2025-04-09 23:16:58'}

Testing Portfolio Rebalancer...
Rebalance result: Portfolio Total Value: $100000.00
Target Weight per Stock: 0.3333 (33.33%)

Current Portfolio Status:
  AAPL: $50000.00 (50.00% vs target 33.33%)
  TSLA: $30000.00 (30.00% vs target 33.33%)
  GOOGL: $20000.00 (20.00% vs target 33.33%)

Recommended Actions:
  AAPL: SELL 83.82 shares ($16666.67) to adjust from 50.00% to 33.33%
  TSLA: BUY 12.25 shares ($3333.33) to adjust from 30.00% to 33.33%
  GOOGL: BUY 84.01 shares ($13333.33) to adjust from 20.00% to 33.33%

Testing Market Trend Analyzer...
Market trends: Market Trends Over the Past Week:

S&P 500:
  L

ERROR:yfinance:$MSFT
: possibly delisted; no price data found  (period=2d) (Yahoo error = "No data found, symbol may be delisted")



Observation: {'symbol': 'MSFT\n', 'current_price': 390.49, 'daily_change': None, 'percent_change': None, 'previous_close': None, 'currency': 'USD', 'company_name': 'Microsoft Corporation', 'timestamp': '2025-04-09 23:17:27'}
Thought:Thought: Now that I have the current price of MSFT, I can think about what to do next. I could use this information to update my portfolio analysis.

Action: None

Action Input: None

Observation: None is not a valid tool, try one of [StockPriceLookup, PortfolioRebalancer, MarketTrendAnalyzer].
Thought:Thought: I'm not sure why I tried to perform a non-existent action again. I think I should stop doing that and actually use the tools provided. Let me think about what to do next.

Action: MarketTrendAnalyzer

Action Input: None

Observation: Market Trends Over the Past Week:

S&P 500:
  Last Close: $5456.90
  Weekly Change: up 1.12% ($60.38)
  Volatility: 6.53%
  Range: $4835.04 - $5499.53

Dow Jones Industrial Average:
  Last Close: $40608.45
  Weekly Chan